In [19]:
import sys
import os
import random
import networkx as nx
sys.path.append(os.path.join(os.getcwd(), "src"))
from src.model import TrafficModel
from car import CarAgent


model = TrafficModel(50, num_intersections=5, num_borders=20, )

In [20]:
import pyoptinterface as poi
from pyoptinterface import highs

In [88]:
opt_model = highs.Model()

In [89]:
possible_lanes = ['intersection_4', 'intersection_7', 'intersection_1']
print(possible_lanes)

time = range(0, 11)
print(time)

cars_at_light = {
                    0:{'intersection_4': 0, 'intersection_7': 0, 'intersection_1': 0}, # Löst Kaltstartproblem; bildet auch realität ab, weil initial keine Autos an ampel
                    1:{'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10},
                    2:{'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10},
                    3:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    4:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    5:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    6:{'intersection_4': 3, 'intersection_7': 7, 'intersection_1': 1},
                    7:{'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, # Längere Betrachtung
                    8:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    9:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                    10:{'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1},
                 }
print(cars_at_light)

['intersection_4', 'intersection_7', 'intersection_1']
range(0, 11)
{0: {'intersection_4': 0, 'intersection_7': 0, 'intersection_1': 0}, 1: {'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, 2: {'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, 3: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 4: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 5: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 6: {'intersection_4': 3, 'intersection_7': 7, 'intersection_1': 1}, 7: {'intersection_4': 5, 'intersection_7': 1, 'intersection_1': 10}, 8: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 9: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}, 10: {'intersection_4': 1, 'intersection_7': 10, 'intersection_1': 1}}


In [90]:
# Entscheidungsvariablen
lanes = opt_model.add_variables(time, possible_lanes, domain=poi.VariableDomain.Binary)

Running HiGHS 1.9.0 (git hash: 66f735e): Copyright (c) 2024 HiGHS under MIT licence terms


In [ ]:
# Restriktion: jede Ampel kann nur exakt eine open_lane haben
for tick in time:
    opt_model.add_linear_constraint(poi.quicksum(lanes[tick, lane] for lane in possible_lanes ), poi.Eq, 1)

# Restriktion: jede Ampel kann nur alle 5 Ticks die open_lane ändern (cooldown)
# for lane in possible_lanes:
    # opt_model.add_linear_constraint(poi.quicksum(((lanes[tick-1, lane] - lanes[tick, lane]) * (lanes[tick-1, lane] - lanes[tick, lane])) for tick in time[1:]), poi.Leq, 1.0) # Glaube hier fehl[t/en] Loops

for decision in lanes:
    opt_model.add_linear_constraint(poi.quicksum(((lanes[i, decision[1]] - lanes[(i + 1), decision[1]]) * (lanes[i, decision[1]] - lanes[(i + 1), decision[1]])) for i in range(max(0, (decision[0] - 5)), decision[0])), poi.In, 1.0)


In [87]:
for decision in lanes:
    print(poi.quicksum(((lanes[i, decision[1]] - lanes[(i + 1), decision[1]]) * (lanes[i, decision[1]] - lanes[(i + 1), decision[1]])) for i in range(max(0, (decision[0] - 5)), decision[0])), poi.Leq, 1.0)


<pyoptinterface._src.core_ext.ExprBuilder object at 0x13ccaba50> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8210> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8170> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8210> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8170> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8210> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8170> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8210> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8170> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object at 0x13cca8210> ConstraintSense.LessEqual 1.0
<pyoptinterface._src.core_ext.ExprBuilder object a

In [64]:
for decision in lanes:
    print(f"Current Decision: {decision}")
    [print(f"Previous Decision: {lanes[i, decision[1]]}") for i in range(max(0, (decision[0] - 5)), decision[0])]
    print("\n")

Current Decision: (0, 'intersection_4')


Current Decision: (0, 'intersection_7')


Current Decision: (0, 'intersection_1')


Current Decision: (1, 'intersection_4')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc132d0>


Current Decision: (1, 'intersection_7')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc13930>


Current Decision: (1, 'intersection_1')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc13a70>


Current Decision: (2, 'intersection_4')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc132d0>
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc13950>


Current Decision: (2, 'intersection_7')
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc13930>
Previous Decision: <pyoptinterface._src.core_ext.VariableIndex object at 0x13cc13890>


Current Decision: (2, 'intersection_1')
Previous Decision: <py

In [92]:
# Zielfunktion
objective = poi.quicksum(poi.quicksum(lanes[tick, lane] * cars_at_light[tick][lane] for lane in possible_lanes) for tick in time)
opt_model.set_objective(objective, poi.ObjectiveSense.Maximize)

Hessian has dimension 33 but no nonzeros, so is ignored


In [93]:
opt_model.optimize()

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
11 rows, 33 cols, 33 nonzeros  0s
0 rows, 7 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   97              97                 0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      97
  Dual bound        97
  Gap           

In [94]:
# Glaub nicht dass der code funktioniert

for tick in time:
    for lane in possible_lanes:
        if opt_model.get_value(lanes[0, lane]) > 0.1:
            print(f"{tick}:{lane}")

0:intersection_4
1:intersection_4
2:intersection_4
3:intersection_4
4:intersection_4
5:intersection_4
6:intersection_4
7:intersection_4
8:intersection_4
9:intersection_4
10:intersection_4


In [95]:
# Entscheidung pro Zeit und Lane

for lane in lanes:
    print(f"Time: {lane[0]}; Intersection: {lane[1]}; Decision: {opt_model.get_value(lanes[lane])}")

Time: 0; Intersection: intersection_4; Decision: 1.0
Time: 0; Intersection: intersection_7; Decision: 0.0
Time: 0; Intersection: intersection_1; Decision: 0.0
Time: 1; Intersection: intersection_4; Decision: 0.0
Time: 1; Intersection: intersection_7; Decision: 0.0
Time: 1; Intersection: intersection_1; Decision: 1.0
Time: 2; Intersection: intersection_4; Decision: 0.0
Time: 2; Intersection: intersection_7; Decision: 0.0
Time: 2; Intersection: intersection_1; Decision: 1.0
Time: 3; Intersection: intersection_4; Decision: 0.0
Time: 3; Intersection: intersection_7; Decision: 1.0
Time: 3; Intersection: intersection_1; Decision: 0.0
Time: 4; Intersection: intersection_4; Decision: 0.0
Time: 4; Intersection: intersection_7; Decision: 1.0
Time: 4; Intersection: intersection_1; Decision: 0.0
Time: 5; Intersection: intersection_4; Decision: 0.0
Time: 5; Intersection: intersection_7; Decision: 1.0
Time: 5; Intersection: intersection_1; Decision: 0.0
Time: 6; Intersection: intersection_4; Decisio

In [96]:
# Entscheidungshistorie; Spalte = Ampel, Zeile gleich Zeitpunkt

decision_matrix = [opt_model.get_value(lanes[lane]) for lane in lanes]
decision_matrix_time = [decision_matrix[i:i+3] for i in range(0, len(decision_matrix), 3)]
decision_matrix_time

[[1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0]]

In [97]:
# Transponierte Entscheidungshistorie

decision_matrix_light = list(map(list, zip(*decision_matrix_time)))
decision_matrix_light

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0],
 [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]

In [98]:
# Cooldown restriktion

for light in decision_matrix_light: # Historie von Entscheidung pro Ampel
    for i in range(len(light)): 
        decision_history = [0] * max(0, 5 - i) + light[max(0, i-5):i] # Entscheidungen von t-5 bis t=0; Wenn vorher keine Entscheidung gab, dann führende 0; Zu erkennen an int 0 vs. float 0.0
        block_status = 0 # Bei 0 darf geöffnet werden, bei 1 ist cooldown aktiv, bei größer 1 gabs nen fehler in der entscheidung
        for j in range(1, (len(decision_history) - 1)):
            block_status += (decision_history[j-1] - decision_history[j]) ** 2 # Berechnen von @mxrio Loop (** ist exponent)
        print(decision_history)
        print(block_status)
        if block_status == 0:
            print("Light can switch")
        elif block_status == 1:
            print("Light is cooldown blocked")
        elif block_status > 1:
            print("Forbidden Action")

[0, 0, 0, 0, 0]
0
Light can switch
[0, 0, 0, 0, 1.0]
0
Light can switch
[0, 0, 0, 1.0, 0.0]
1.0
Light is cooldown blocked
[0, 0, 1.0, 0.0, 0.0]
2.0
Forbidden Action
[0, 1.0, 0.0, 0.0, 0.0]
2.0
Forbidden Action
[1.0, 0.0, 0.0, 0.0, 0.0]
1.0
Light is cooldown blocked
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0, 0, 0, 0, 0]
0
Light can switch
[0, 0, 0, 0, 0.0]
0
Light can switch
[0, 0, 0, 0.0, 0.0]
0.0
Light can switch
[0, 0, 0.0, 0.0, 0.0]
0.0
Light can switch
[0, 0.0, 0.0, 0.0, 1.0]
0.0
Light can switch
[0.0, 0.0, 0.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[0.0, 0.0, 1.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[0.0, 1.0, 1.0, 1.0, 1.0]
1.0
Light is cooldown blocked
[1.0, 1.0, 1.0, 1.0, 0.0]
0.0
Light can switch
[1.0, 1.0, 1.0, 0.0, 1.0]
1.0
Light is cooldown blocked
[1.0, 1.0, 0.0, 1.0, 1.0]
2

In [99]:
print(f"Lane: {opt_model.get_obj_value()}")

Lane: 97.0
